<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/custom-prompts-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom prompts with Vectara

Vectara includes many powerful features for building RAG pipelines, and in this notebook we want to demonstrate one of them - using custom prompts, which allow for better control of the LLM responses. For example you can use custom prompts to change the style and tone of the response.

In [1]:
import requests
import json
import re
import os
from urllib.parse import quote
from IPython.display import display, Markdown

## Query helper functions

Before we start, here is some Python code that is helpful to abstract some of the fine details of calling the Vectara API.

The `VectaraQuery` class simplifies running a query against a Vectara corpus.
Note that in addition to the expected `customer_id`, `corpus_id` and `api_key`, it has two additional arguments in the constructor: `prompt_name` and `prompt_text`, which are the name of the prompt we would use and the text of the prompt when we use custom prompts. 

In [2]:
class VectaraQuery():
    def __init__(self, api_key: str, customer_id: str, corpus_id: str, prompt_name: str = None, prompt_text: str = None):
        self.customer_id = customer_id
        self.corpus_id = corpus_id
        self.api_key = api_key
        self.prompt_name = prompt_name if prompt_name else "vectara-summary-ext-24-05-sml"
        self.prompt_text = prompt_text

    def get_body(self, query_str: str):
        corpora_key_list = [{
                'customer_id': self.customer_id, 'corpus_id': self.corpus_id, 'lexical_interpolation_config': {'lambda': 0.005}
            }
        ]
        body = {
            'query': [
                { 
                    'query': query_str,
                    'start': 0,
                    'numResults': 50,
                    'corpusKey': corpora_key_list,
                    'context_config': {
                        'sentences_before': 2,
                        'sentences_after': 2,
                        'start_tag': "%START_SNIPPET%",
                        'end_tag': "%END_SNIPPET%",
                    },
                    'rerankingConfig':
                    {
                        'rerankerId': 272725719,
                    },
                    'summary': [
                        {
                            'responseLang': 'eng',
                            'maxSummarizedResults': 7,
                            'summarizerPromptName': self.prompt_name,
                        }
                    ]
                } 
            ]
        }
        if self.prompt_text:
            body['query'][0]['summary'][0]['promptText'] = self.prompt_text
        return body

    def get_headers(self):
        return {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "customer-id": self.customer_id,
            "x-api-key": self.api_key,
            "grpc-timeout": "60S"
        }

    def submit_query(self, query_str: str):

        endpoint = f"https://api.vectara.io/v1/query"
        body = self.get_body(query_str)

        response = requests.post(endpoint, data=json.dumps(body), verify=True, headers=self.get_headers())    
        if response.status_code != 200:
            print(f"Query failed with code {response.status_code}, reason {response.reason}, text {response.text}")
            return "Sorry, something went wrong in my brain. Please try again later."

        res = response.json()
        
        top_k = 10
        summary = res['responseSet'][0]['summary'][0]['text']

        return summary

In this demo I've used the same corpus that includes all the text from Richard Feynman's lectures, as show for example in this [demo application](https://askfeynman.demo.vectara.com/). Let's read from our environment the `customer_id`, `corpus_id` and `api_key` we want to use for accessing Vectara. 

In [3]:
api_key = os.environ['VECTARA_API_KEY']
customer_id = os.environ['VECTARA_CUSTOMER_ID']
corpus_id = os.environ['VECTARA_CORPUS_ID']

## Using custom prompts

Now let's go to custom prompts, a powerful feature of Vectara, with which you can control and customize your prompts to fit your use-case. This can be helpful in generating responses that are in a certain style or form, as well as change the behavior of the summarizer to perform other actions.

Let's look at a few examples. First we are going to ask a simple physics question: "what is an atom?"

In [4]:
vq = VectaraQuery(api_key, customer_id, corpus_id, prompt_name='vectara-summary-ext-24-05-med-omni')
response = vq.submit_query("what is an atom?")
display(Markdown(response))

An atom is a fundamental unit of matter, consisting of a nucleus and electrons. The nucleus, which contains most of the atom's mass, is extremely small compared to the entire atom, with a diameter of about 10^-13 cm, while the atom itself is about 10^-8 cm in diameter [3]. Atoms are in perpetual motion, attracting each other when slightly apart and repelling when squeezed together [5]. They can exist in different energy states and can become excited, but they eventually return to a lower energy state by interacting with the electromagnetic field [2]. The behavior of atoms is governed by quantum mechanics, which differs significantly from the classical Newtonian physics that applies to larger objects [4].

And now let's try a few custom prompts. We'll start with a simple one:

In [5]:
prompt1 = '''
[
  {"role": "system", "content": "You are a helpful search assistant. 
                                 Make sure you base your response only on the search results provided."},
  #foreach ($qResult in $vectaraQueryResults)
     {"role": "user", "content": "Give me the $vectaraIdxWord[$foreach.index] search result."},
     {"role": "assistant", "content": "${qResult.getText()}" },
  #end
  {"role": "user", "content": "Generate a summary for the query '${vectaraQuery}' based on the above results."}
]
'''

vq = VectaraQuery(api_key, customer_id, corpus_id, 
                  prompt_name = 'vectara-summary-ext-24-05-med-omni', 
                  prompt_text = prompt1)
response = vq.submit_query("what is an atom?")
display(Markdown(response))

An atom is a fundamental unit of matter, consisting of a nucleus and electrons. The nucleus, which contains most of the atom's mass, is extremely small compared to the overall size of the atom. Atoms are about 10^-8 cm in diameter, while the nucleus is about 10^-13 cm. Electrons orbit the nucleus, prevented from collapsing into it by the uncertainty principle, which dictates that knowing an electron's precise position would require it to have a large, uncertain momentum.

Atoms are in perpetual motion, attracting each other when slightly apart and repelling when squeezed together. This atomic behavior forms the basis of the atomic hypothesis, which posits that all matter is composed of these tiny particles. Atoms can also exist in excited states, where they have higher energy levels, but they tend to return to lower energy states by interacting with electromagnetic fields.

In certain conditions, atoms can lose electrons and become ions. The number of ions, neutral atoms, and electrons in a given volume depends on factors like density and temperature. Understanding atoms and their interactions is crucial for explaining the fundamental laws of physics, including quantum mechanics, which governs atomic behavior and differs significantly from Newton's laws that apply to larger-scale objects.

Now let's play with the prompt to make this more interesting: can we ask our RAG pipeline to respond in bullet points?

In [6]:
prompt2 = '''
[
  {"role": "system", "content": "You are a helpful search assistant, with expertise in physics. 
                                 You respond in bullet points."},
  #foreach ($qResult in $vectaraQueryResults)
     {"role": "user", "content": "Give me the $vectaraIdxWord[$foreach.index] search result."},
     {"role": "assistant", "content": "${qResult.getText()}" },
  #end
  {"role": "user", "content": "Generate a summary for the query '${vectaraQuery}' based on the above results."}
]
'''

vq = VectaraQuery(api_key, customer_id, corpus_id, 
                  prompt_name = 'vectara-summary-ext-24-05-med-omni', 
                  prompt_text = prompt2)
response = vq.submit_query("what is an atom?")
display(Markdown(response))

- An atom is a fundamental unit of matter, with a typical diameter of about (10^{-8}) cm.
- Atoms consist of a nucleus, which is much smaller (about (10^{-13}) cm in diameter) and contains most of the atom's mass.
- Electrons orbit the nucleus, and their behavior is governed by quantum mechanics, which differs significantly from classical Newtonian physics.
- Atoms are in constant motion, attracting each other when slightly apart and repelling when too close.
- The atomic hypothesis suggests that all matter is composed of these small particles, which are in perpetual motion.
- Atoms can become ions if they lose or gain electrons, a process influenced by external energy and the surrounding environment.
- Understanding atoms involves considering their interactions, energy states, and the principles of quantum mechanics.

Last one - Explain atoms to me like I'm a 5-year-old:

In [7]:
prompt3 = '''
[
  {"role": "system", "content": "You are a helpful search assistant, with expertise in physics. 
                                 Respond in a way that a five year old can understand."},
  #foreach ($qResult in $vectaraQueryResults)
     {"role": "user", "content": "Give me the $vectaraIdxWord[$foreach.index] search result."},
     {"role": "assistant", "content": "${qResult.getText()}" },
  #end
  {"role": "user", "content": "Generate a summary for the query|'${vectaraQuery}' based on the above results."}
]
'''

vq = VectaraQuery(api_key, customer_id, corpus_id, 
                  prompt_name = 'vectara-summary-ext-24-05-med-omni',
                  prompt_text = prompt3)
response = vq.submit_query("what is an atom?")
display(Markdown(response))

An atom is a tiny particle that makes up everything around us. Imagine if you had a super-duper magnifying glass and looked at anything, like a toy or a drop of water, you'd see these tiny atoms. They're so small that if an atom were the size of a big room, its center, called the nucleus, would be just a tiny speck! Atoms are always moving, and they like to stick together to make all the things we see. They have a center with most of their weight, and little parts called electrons that zoom around the center. Even though they're super tiny, understanding atoms helps us learn a lot about how the world works!

There's a lot more you can do with custom prompts and we at Vectara are curious to hear what are you building with these powerful capabilities, so please don't hesistate to share your success with us in the [discussion forums](https://discuss.vectara.com/) or our [Discord community server](https://discord.com/invite/GFb8gMz6UH)